## How to use Scorers
This demo includes how to use scorers to get same size scores for each score. 

In [1]:
import torch
from torch.autograd import Variable
from torch.nn import Parameter
import sys
sys.path.insert(0,"../")
from pytorchpruner.scorers import magnitudeScorer,gradientScorer,hessianScorer

TOLERANCE = 1e-5

In [2]:
# We have 4 random parameters
w = Parameter(torch.rand(2,2))
print(w)

def L2d(w):
    '''
    A custom loss function
        w00 w01
        w10 w11
    '''    
    return (w[1,0]**2)*w[1,1]+4*(w[0,0]**3)*w[1,0]

def G2d(w):
    #Jacobian of the L(w)
    wd = w.data
    return torch.Tensor([
                         [12*(wd[0,0]**2)*wd[1,0],
                          0],
                         [2*wd[1,0]*wd[1,1]+4*(wd[0,0]**3),
                          wd[1,0]**2]
                        ])

#torch.autograd
loss_val = L2d(w)
grad_score = gradientScorer(loss_val, w)
print(grad_score)
#emprical gradient
correct_score = G2d(w)
# print(correct_score)
print((grad_score-correct_score).abs().sum() < TOLERANCE)


Parameter containing:
 0.3538  0.9319
 0.1769  0.5440
[torch.FloatTensor of size 2x2]


 0.2656  0.0000
 0.3695  0.0313
[torch.FloatTensor of size 2x2]

True


In [3]:
hessian_score = hessianScorer(loss_val,w)

print(hessian_score)
def H2dScore(w):
    #Hessian of the L2(w)
    wd=w.data
    gw12 = 2*wd[1,0]
    gw13 = 12*(wd[0,0]**2)
    gw23 = 0
    gw11 = 2*wd[1,1]
    gw22 = 0
    gw33 = 24*wd[0,0]*wd[1,0]
    ## x3 0
    ## x1 x2
    return torch.Tensor([[gw33+0+gw13+gw23, #0,0,: #x3 with others
                       0+0+0+0], #0,1,:
                      [gw13+0+gw11+gw12, #1,0,: #x1 with others
                       gw23+0+gw12+gw22]]) #1,1,: #x2 with others

correct_score = H2dScore( w)
print((correct_score-hessian_score).abs().sum() < TOLERANCE)


 3.0034  0.0000
 2.9434  0.3537
[torch.FloatTensor of size 2x2]

True
